# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [29]:
document = ET.parse( './data/mondial_database.xml' )



In [38]:
import pandas as pd

country_infant_mortality = []
for country in document.iterfind('country'):
    if country.find('infant_mortality') is not None:
        country_infant_mortality.append([country.find('name').text,country.find('infant_mortality').text])

# print(country_infant_mortality)

df_infant_mortality = pd.DataFrame(country_infant_mortality, columns = ['Country', 'Infant_Mortality'])
df_infant_mortality.Infant_Mortality = df_infant_mortality.Infant_Mortality.astype(float)
df_infant_mortality.sort_values(by = 'Infant_Mortality').head(10)


,Country,Infant_Mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [44]:

City_Pop_List = []

for country in document.iterfind('country'):
    for city in country.getiterator('city'):
        for population in city.iterfind('population'):
            last_pop = int(population.text)
        City_Pop_List.append([city.findtext('name'), last_pop])
        
df_city_population = pd.DataFrame(City_Pop_List, columns = ['City', 'Population'])
df_city_population.sort_values(by = 'Population', ascending=False).head(10)

,City,Population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


In [68]:
ethnic_groups = {}

for country in document.iterfind('country'):
    population_list=[]
    for population in country.getiterator('population'):
       population_list.append(int(population.text))
    last_population = max(population_list)
    for ethnic in country.iterfind('ethnicgroup'):
        if ethnic.text in ethnic_groups:
            ethnic_groups[ethnic.text] += last_population*float(ethnic.attrib['percentage'])/100
        else:
            ethnic_groups[ethnic.text] = last_population*float(ethnic.attrib['percentage'])/100        

df_ethnic_groups = pd.DataFrame.from_dict(ethnic_groups, orient='index')
pd.options.display.float_format = '{:20,.2f}'.format
df_ethnic_groups.columns = ['Population']
df_ethnic_groups.sort_values(by='Population', ascending=False).head(10)

,Population
Han Chinese,"1,245,058,800.00"
Indo-Aryan,"871,815,583.44"
European,"494,939,515.65"
African,"318,359,698.05"
Dravidian,"302,713,744.25"
Mestizo,"157,855,273.00"
Bengali,"146,776,916.72"
Russian,"136,866,550.64"
Japanese,"127,289,007.89"
Malay,"121,993,620.27"


In [102]:
rivers_list=[]
for river in document.iterfind('river'):
    country = river.attrib['country']
    name = river.findtext('name')
    length = river.findtext('length')
    if length is None:
      length = 0
    else:
       length = float(length) 
    rivers_list.append([country, name, length])
df_rivers = pd.DataFrame(rivers_list, columns = ['Country', 'Name', 'Length'])
df_rivers.sort_values(by='Length', ascending=False).head(10)

,Country,Name,Length
174,CO BR PE,Amazonas,"6,448.00"
137,CN,Jangtse,"6,380.00"
136,CN,Hwangho,"4,845.00"
123,R,Lena,"4,400.00"
205,RCB ZRE,Zaire,"4,374.00"
138,CN LAO THA K VN,Mekong,"4,350.00"
115,R KAZ CN,Irtysch,"4,248.00"
186,RMM RN WAN RG,Niger,"4,184.00"
160,USA,Missouri,"4,130.00"
119,R,Jenissej,"4,092.00"


In [114]:
lake_list=[]
for lake in document.iterfind('lake'):
    country = lake.attrib['country']
    name = lake.findtext('name')
    area = lake.findtext('area')
    if area is None:
      area = 0
    else:
      area = float(area) 
    lake_list.append([country, name, area])
df_lakes = pd.DataFrame(lake_list, columns = ['Country', 'Name', 'Area'])
df_lakes.sort_values(by='Area', ascending=False).head(10)

,Country,Name,Area
54,R AZ KAZ IR TM,Caspian Sea,"386,400.00"
109,CDN USA,Lake Superior,"82,103.00"
81,EAT EAK EAU,Lake Victoria,"68,870.00"
106,CDN USA,Lake Huron,"59,600.00"
108,USA,Lake Michigan,"57,800.00"
47,IL JOR WEST,Dead Sea,"41,650.00"
83,ZRE Z BI EAT,Lake Tanganjika,"32,893.00"
98,CDN,Great Bear Lake,"31,792.00"
43,R,Ozero Baikal,"31,492.00"
89,MW MOC EAT,Lake Malawi,"29,600.00"


In [124]:
airport_list=[]
for airport in document.iterfind('airport'):
    country = airport.attrib['country']
    name = airport.findtext('name')
    height = airport.findtext('elevation')
    if height is None:
      height = 0
    else:
      try:
        height=float(height)
      except ValueError:
        height = 0
    airport_list.append([country, name, height])
df_airports = pd.DataFrame(airport_list, columns = ['Country', 'Name', 'Elevation'])
df_airports.sort_values(by='Elevation', ascending=False).head(10)

,Country,Name,Elevation
80,BOL,El Alto Intl,"4,063.00"
219,CN,Lhasa-Gonggar,"4,005.00"
241,CN,Yushu Batang,"3,963.00"
813,PE,Juliaca,"3,827.00"
815,PE,Teniente Alejandro Velasco Astete Intl,"3,311.00"
82,BOL,Juana Azurduy De Padilla,"2,905.00"
334,EC,Mariscal Sucre Intl,"2,813.00"
805,PE,Coronel Fap Alfredo Mendivil Duarte,"2,719.00"
807,PE,Mayor General FAP Armando Revoredo Iglesias Ai...,"2,677.00"
692,MEX,Licenciado Adolfo Lopez Mateos Intl,"2,581.00"
